# Multichannel operation with EIS

This notebook is an example in which [external potentiostats](http://zahner.de/products/external-potentiostats.html) such as PP2x2 and XPOT2 are controlled both standalone and as an EPC device on a [Zennium series instrument](http://zahner.de/products/electrochemical-workstation.html). Up to 16 external potentiostats can share one Zennium to use it for impedance measurements for example.

This notebook cannot be executed and has been created only for documentation and explanation of the source code. The epc_scpi_handler.py file contains the required classes for this example.

Knowledge of all other notebooks of this repository, the [Remote2 manual](http://zahner.de/pdf/Remote2.pdf) and control with the [Thales Remote Python library](https://github.com/Zahner-elektrik/Thales-Remote-Python) for Remote2 is assumed as known.

For this example a Zennium with [EPC card](http://zahner.de/products/addon-cards/epc42.html) is necessary, to this EPC card the external potentiostats must be connected with the appropriate cable, this can be read in the [manual](http://zahner.de/files/power_potentiostats.pdf). A maximum of 4 potentiostats per card and a maximum of 4 cards are possible.  
The external potentiostats and the Zennium must also be connected to the computer via USB cable separately.

In this example, a PP242 and an XPOT2 are used to cyclically charge and discharge capacitors, and after a defined number of cycles, impedance is measured with the Zennium via the EPC interface.

Each external potentiostat needs its own thread in which the measurement sequence is programmed.  
The shared Zennium is synchronized with a [Lock object](https://docs.python.org/3/library/threading.html#lock-objects).
<div class="alert alert-block alert-warning">
    <b>Only one potentiostat, i.e. thread, can access the Zennium at a time.</b>
</div>

In [ ]:
from epc_scpi_handler import EpcScpiHandlerFactory,EpcScpiHandler

from thales_remote.script_wrapper import PotentiostatMode
from zahner_potentiostat.scpi_control.datahandler import DataManager
from zahner_potentiostat.display.onlinedisplay import OnlineDisplay

import threading
from datetime import datetime

from jupyter_utils import executionInNotebook, notebookCodeToPython

# Help function for naming

This function simplifies the naming of the cells/channels.

In [ ]:
def getFileName(channel,cycle):
    string = str(datetime.now().time())
    string = string.replace(":","")
    string = string.replace(",","")
    string = string.replace(".","")
    string = "channel" + str(channel) + "_cycle" + str(cycle) # + string
    return string

# Thread for channel 1

The following function is executed as a thread for channel one.
The same function could also be executed as a thread for each channel, but here different sequences are to be measured with each external potentiostat, therefore two different functions are used.

Only the first thread is explained in more detail, the second thread is only slightly different from the first thread.

The threads unfortunately have to be defined before the main function, where the initialization of the different devices takes place and EpcScpiHandler objects are created for each external potentiostat.

An EpcScpiHandler object is then passed to the respective thread as a parameter, so that the thread can work with this object and the measurement process can be programmed.  
Using this concept it is also possible to use the same function several times as thread, if all channels should do the same. For this example channel1Thread() can be passed a number for the naming of the measurement results.

In [ ]:
def channel1Thread(deviceHandler, channel = 0):
    deviceHandler.scpiInterface.setMaximumTimeParameter(15)
    
    for i in range(3):
        filename = getFileName(channel = channel, cycle = i)      

First the device is in SCPI standalone mode and a capacitor is charged and discharged 2 times in standalone mode.

During the measurement, the online display is also started for visualization of voltage and current.

In [ ]:
        onlineDisplay = OnlineDisplay(deviceHandler.scpiInterface.getDataReceiver())
        
        deviceHandler.scpiInterface.measureOCVScan()
        
        for n in range(2):
            deviceHandler.scpiInterface.measureCharge(current = 1e-3,
                                                      stopVoltage = 2,
                                                      maximumTime = "5 min")
            
            deviceHandler.scpiInterface.measureDischarge(current = -1e-3,
                                                         stopVoltage = 0.5,
                                                         maximumTime = "5 min")

After cycling, the EPC mode must be activated in order to control the potentiostat as an EPC device with the Zennium. For this purpose the lock must be aquired.  
In order to be able to continue measuring the OCP, the lock is acquired with blocking = False, so if the Zennium is occupied and **acquireSharedZennium()** returns False, the OCP is measured again for 15 seconds.  
This is repeated until the Zennium is no longer occupied by another channel.

In [ ]:
        deviceHandler.scpiInterface.setMaximumTimeParameter(15)    
        while deviceHandler.acquireSharedZennium(blocking = False) == False:
            deviceHandler.scpiInterface.measureOCVScan()

Now that the Zennium is reserved for this measurement, the measurement data is saved as text and the online display is closed.

In [ ]:
        dataManager = DataManager(deviceHandler.scpiInterface.getDataReceiver())
        dataManager.saveDataAsText(filename + ".txt")
        
        onlineDisplay.close()
        del onlineDisplay

To use the external potentiostat as an EPC device, the device must be switched to EPC mode as shown below.

The change between SCPI and EPC interface must be initiated by the currently active controller, it is not possible to "get back" the control via SCPI in EPC mode.

In [ ]:
        deviceHandler.switchToEPC()

From now on the Zennium is controlled by Remote2 and the [Thales-Remote-Python library](https://github.com/Zahner-elektrik/Thales-Remote-Python), the switchToEPC() function automatically selects the EPC channel to which the device is connected.

In [ ]:
        deviceHandler.sharedZenniumInterface.setEISNaming("individual")
        deviceHandler.sharedZenniumInterface.setEISOutputPath(r"C:\THALES\temp\multichannel")
        deviceHandler.sharedZenniumInterface.setEISOutputFileName(filename)
        
        deviceHandler.sharedZenniumInterface.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
        deviceHandler.sharedZenniumInterface.setAmplitude(10e-3)
        deviceHandler.sharedZenniumInterface.setPotential(0)
        deviceHandler.sharedZenniumInterface.setLowerFrequencyLimit(100)
        deviceHandler.sharedZenniumInterface.setStartFrequency(500)
        deviceHandler.sharedZenniumInterface.setUpperFrequencyLimit(1000)
        deviceHandler.sharedZenniumInterface.setLowerNumberOfPeriods(5)
        deviceHandler.sharedZenniumInterface.setLowerStepsPerDecade(2)
        deviceHandler.sharedZenniumInterface.setUpperNumberOfPeriods(20)
        deviceHandler.sharedZenniumInterface.setUpperStepsPerDecade(5)
        deviceHandler.sharedZenniumInterface.setScanDirection("startToMax")
        deviceHandler.sharedZenniumInterface.setScanStrategy("single")
        
        deviceHandler.sharedZenniumInterface.measureEIS()

After the measurement, the switch back to standalone SCPI operation must be initiated and the Zennium must be enabled to be used by another channel.

In [ ]:
        deviceHandler.switchToSCPIAndReleaseSharedZennium()
           
    return

# Thread for channel 2

The following function is executed as a thread for channel 2.

In [ ]:
def channel2Thread(deviceHandler):
    deviceHandler.scpiInterface.setMaximumTimeParameter(15)
    
    for i in range(2):
        filename = getFileName(channel = 2, cycle = i)
               
        onlineDisplay = OnlineDisplay(deviceHandler.scpiInterface.getDataReceiver())
        
        deviceHandler.scpiInterface.measureOCVScan()
        
        for n in range(2):
            deviceHandler.scpiInterface.measureCharge(current = 4,
                                                      stopVoltage = 1,
                                                      maximumTime = "5 min")
            
            deviceHandler.scpiInterface.measureDischarge(current = -4,
                                                         stopVoltage = 0.6,
                                                         maximumTime = "5 min")
        
        dataManager = DataManager(deviceHandler.scpiInterface.getDataReceiver())
        dataManager.saveDataAsText(filename + ".txt")
        
        onlineDisplay.close()
        del onlineDisplay

In this example we simply wait without measurement until the potentiostat is free, therefore no parameters are necessary for acquireSharedZennium().  
The potentiostat should have been switched off before.

In [ ]:
        deviceHandler.acquireSharedZennium()              
        deviceHandler.switchToEPC()

        deviceHandler.sharedZenniumInterface.setEISNaming("individual")
        deviceHandler.sharedZenniumInterface.setEISOutputPath(r"C:\THALES\temp\multichannel")
        deviceHandler.sharedZenniumInterface.setEISOutputFileName(filename)
        
        deviceHandler.sharedZenniumInterface.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
        deviceHandler.sharedZenniumInterface.setAmplitude(10e-3)
        deviceHandler.sharedZenniumInterface.setPotential(0)
        deviceHandler.sharedZenniumInterface.setLowerFrequencyLimit(100)
        deviceHandler.sharedZenniumInterface.setStartFrequency(500)
        deviceHandler.sharedZenniumInterface.setUpperFrequencyLimit(1000)
        deviceHandler.sharedZenniumInterface.setLowerNumberOfPeriods(5)
        deviceHandler.sharedZenniumInterface.setLowerStepsPerDecade(2)
        deviceHandler.sharedZenniumInterface.setUpperNumberOfPeriods(20)
        deviceHandler.sharedZenniumInterface.setUpperStepsPerDecade(5)
        deviceHandler.sharedZenniumInterface.setScanDirection("startToMax")
        deviceHandler.sharedZenniumInterface.setScanStrategy("single")
        
        deviceHandler.sharedZenniumInterface.measureEIS()
        
        deviceHandler.switchToSCPIAndReleaseSharedZennium()
               
    
    return

# Initialization of the devices

Before the two threads with the measurement tasks are executed, the device management objects of type EpcScpiHandler are created with the EpcScpiHandlerFactory class.

The Zennium could also be connected to another computer via USB and you can control the Zennium over network, for this the default parameter *shared_zennium_target = "localhost"* of the constructor of EpcScpiHandlerFactory must be overwritten with the corresponding IP address.
The external potentiostats can currently only be controlled via USB.

In [ ]:
if __name__ == '__main__':    
    handlerFactory = EpcScpiHandlerFactory()

With the method createEpcScpiHandler of the EpcScpiHandlerFactory object handlerFactory, a new EpcScpiHandler object can be created, which is used to control the devices as explained above.

The method createEpcScpiHandler() has two parameters:  
* **epcChannel:** The number of the EPC channel to which the external potentiostat is connected.
* **serialNumber:** The serial number of the device to uniquely identify it.

In [ ]:
    XPOT2 = handlerFactory.createEpcScpiHandler(epcChannel=1, serialNumber=27000)
    PP242 = handlerFactory.createEpcScpiHandler(epcChannel=4, serialNumber=35000)

After the two devices are initialized, they are passed to the respective threads and the threads are started. Then it only waits for the two threads to finish and then everything is closed.  
For the first thread a channel number is passed, this would be necessary for the naming of the files, if the same function is executed as different threads.

In [ ]:
    channel1ThreadHandler = threading.Thread(target=channel1Thread, args=(XPOT2,1))
    channel2ThreadHandler = threading.Thread(target=channel2Thread, args=(PP242,))
    
    channel1ThreadHandler.start()
    channel2ThreadHandler.start()
    
    channel1ThreadHandler.join()
    channel2ThreadHandler.join()
    
    handlerFactory.closeAll()
    print("finish")

# Deployment of the source code

**The following instruction is not needed by the user.**

It automatically extracts the pure python code from the jupyter notebook to provide it for the user.  
Thus the user does not need jupyter itself and does not have to copy the code manually.

The code is stored in a notebook-like file with the extension .py.

In [ ]:
    if executionInNotebook() == True:
        notebookCodeToPython("ImpedanceMultiCellCycle.ipynb")